In [ ]:
!poetry add langchain-openai langchainhub

In [ ]:
!poetry add faiss-cpu

In [1]:
# 在启动Jupyter的根目录编辑一个 .env ，然后执行下面代码自动加载环境变量
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [ ]:
# 或者手工输入环境变量
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')
os.environ['TAVILY_API_KEY'] = getpass.getpass('TAVILY API Key:')

# 简单的智能体例子

## 工具1：搜索工具

In [2]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()

使用工具之前先看看效果：

In [15]:
search.invoke({"query": "最火的修仙小说"})

[{'url': 'https://www.zhihu.com/question/444289743',
  'content': '第一部是《 紫府仙缘 》，主角就是天生的苟，各种苟，明明实力同等级第一，偏偏要装成弱鸡，在第一轮接触就被秒杀（装死），然后默默苟到最后，一轮爆发干死 鹬蚌 ，坐收渔利。. 这一部讲述了整个大陆的总体的样貌，但留下了数以万计的大坑。. 已完结 ...'},
 {'url': 'https://m.maigoo.com/top/413883.html',
  'content': '大道争锋. 0. 戮仙. 勿扰飞升. 仙本纯良. 我不成仙. 慢慢仙途. 修真界败类. 十大完结修仙小说排行榜，本榜单主要依据各大小说网站（起点中文网、创世中文网、纵横中文网、晋江文学等）的推荐数、收藏数、点击数，网友推荐、百度指数等等情况，并综合参考 ...'},
 {'url': 'https://www.zhihu.com/question/409513352',
  'content': '文笔当然有与《大道争锋》差别不大，坐等完结再看。. 3.《剑徒之路》惰堕. 当初乌贼的看完 《灭运图录》 （仙葫流，强烈推荐，从第二卷开阅读）和《大道争锋》之后，一直想找一个相似世界观和设定的小说（重视心性修炼，配角心志坚定智商较高），然而 ...'},
 {'url': 'https://www.maigoo.com/top/425441.html',
  'content': '2024年最火修仙小说有哪些？目前什么修仙小说人气最高？当下热门的修仙小说越来越多了，除了一些经典的完本修仙小说外，还有许多正在连载的修仙小说，都不知道看什么好。 ... 2024十大热门修仙小说排行榜 人气修仙小说top10 最火的修仙小说推荐 ...'},
 {'url': 'https://zhuanlan.zhihu.com/p/654899257',
  'content': '1. 《凡人修仙传》 - 作者：忘语. 简介：这是一部经典的修仙小说，讲述了主人公方源从一个普通凡人开始修炼，逐渐成为强大修仙者的故事。. 他经历了数次生死考验，探索仙侠世界的奥秘。. 2. 《仙逆》 - 作者：耳根. 简介：故事以普通少年苏名章为主角 ...'}]

## 工具2：向量文本检索

In [17]:
from langchain_community.document_loaders import WebBaseLoader

docs = WebBaseLoader("https://python.langchain.com/docs/langgraph").load()

In [50]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

In [51]:
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [52]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langgraph_search",
    "搜索关于LangGraph的信息。询问任何关于LangGraph的信息，你都必须使用这个工具!",
)

## 创建智能体

## 定义行动部份

In [38]:
tools = [search, retriever_tool]

## 定义思考部份

In [39]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [40]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

In [46]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)

## 定义运行器

In [47]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## 运行智能体

In [49]:
agent_executor.invoke({"input": "你好!"})



> Entering new AgentExecutor chain...
你好！有什么可以帮助你的吗？如果你有任何问题或需要帮助，请随时告诉我。

> Finished chain.
{'output': '你好！有什么可以帮助你的吗？如果你有任何问题或需要帮助，请随时告诉我。', 'messages': [AIMessage(content='你好！有什么可以帮助你的吗？如果你有任何问题或需要帮助，请随时告诉我。')]}


In [43]:
agent_executor.invoke({"input": "langgraph是什么？"})



> Entering new AgentExecutor chain...

Invoking: `langgraph_search` with `{'query': 'What is LangGraph?'}`


ü¶úüï∏Ô∏èLangGraph | ü¶úÔ∏èüîó Langchain

It is inspired by Pregel and Apache Beam.
The current interface exposed is one inspired by NetworkX.The main use is for adding cycles to your LLM application.
Crucially, this is NOT a DAG framework.
If you want to build a DAG, you should just use LangChain Expression Language.Cycles are important for agent-like behaviors, where you call an LLM in a loop, asking it what action to take next.Installation‚Äãpip install langgraphQuick Start‚ÄãHere we will go over an example of creating a simple agent that uses chat models and function calling.

Skip to main contentDocsUse casesIntegrationsGuidesAPIMorePeopleVersioningChangelogContributingTemplatesCookbooksTutorialsYouTubeü¶úÔ∏èüîóLangSmithLangSmith DocsLangServe GitHubTemplates GitHubTemplates HubLangChain HubJS/TS DocsChatSearchGet startedIntroductionInstallationQuickstartSecurityLan

{'input': 'langgraph是什么？',
 'output': 'LangGraph是一个用于构建具有LLM（Language Model）的有状态、多角色应用程序的库，建立在LangChain之上。它扩展了LangChain Expression Language，使其能够以循环方式协调多个链条（或角色）在多个计算步骤中的操作。LangGraph受到Pregel和Apache Beam的启发，主要用于向您的LLM应用程序添加循环。LangGraph不是一个DAG框架，如果您想构建DAG，应该使用LangChain Expression Language。循环对于类似代理的行为非常重要，其中您在循环中调用LLM，询问它下一步应该采取什么行动。'}

In [44]:
agent_executor.invoke({"input": "实现一个langgraph应用需要做哪些步骤?"})



> Entering new AgentExecutor chain...

Invoking: `langgraph_search` with `{'query': 'Steps to implement a LangGraph application'}`


For a walkthrough on how to do that, see this documentationStreaming Tokens‚ÄãSometimes language models take a while to respond and you may want to stream tokens to end users.
For a guide on how to do this, see this documentationPersistence‚ÄãLangGraph comes with built-in persistence, allowing you to save the state of the graph at point and resume from there.
For a walkthrough on how to do that, see this documentationHuman-in-the-loop‚ÄãLangGraph comes with built-in support for human-in-the-loop workflows. This is useful when you want to have a human review the current state before proceeding to a particular node.

print("----")PreviousLangSmith WalkthroughOverviewInstallationQuick StartSet up the toolsSet up the modelDefine the agent stateDefine the nodesDefine the graphUse it!StreamingStreaming Node OutputStreaming LLM TokensWhen to UseExamplesChatAge

{'input': '实现一个langgraph应用需要做哪些步骤?',
 'output': '实现一个LangGraph应用需要完成以下步骤：\n\n1. 安装LangGraph：使用pip install langgraph命令安装LangGraph。\n2. 快速开始：创建一个简单的代理程序，该代理程序使用聊天模型和函数调用。可以参考快速开始文档进行操作。\n3. 设置工具：设置工具，包括模型、代理状态、节点和图。\n4. 使用流式传输：有时语言模型需要一段时间才能响应，可以使用流式传输来向最终用户传递令牌。\n5. 持久性：LangGraph内置持久性功能，允许您保存图的状态并在需要时恢复。\n6. 人在循环中：LangGraph内置支持人在循环工作流程，当您希望在继续到特定节点之前让人员审查当前状态时，这将非常有用。\n\n您可以参考LangGraph的文档和示例来详细了解如何实现LangGraph应用。'}

# 概念

## Schema

LangChain定义了让更容易让智能体工作的几个抽象类。

### AgentAction

这是包装数据的类，包含一个**tool**属性和一个**tool_input**属性。

### AgentFinish

返回最终结果。

### Intermediate Steps

当前智能体返回的中间结果，会作为将来继续执行时的上下文。

## Agent

这是一个决定下一步如何执行的 **chain**，通常会包含大模型、提示语和输出解析等关键部件。

### Agent Inputs

通常是包含 **Intermediate Steps** 的键值对。

### Agent Outputs

通常是 **AgentAction** 或 **AgentFinish** 。

## AgentExecutor

调度 **Agent** 运行时，选择并执行其中包含的 **actions**。

下面是简单的例子：

In [ ]:
next_action = agent.get_action(...)
while next_action != AgentFinish:
    observation = run(next_action)
    next_action = agent.get_action(..., next_action, observation)
return next_action

你还应当考虑：

- 处理选中的 **tool** 不存在的情况
- 处理 **tool** 中的异常
- 处理智能体的输出结果无法映射到 **tool** 调用的情况
- 打印或结合 **langfuse/langsmith** 处理日志和各层级的调试信息

## Tools

**Tool** 的抽象是告诉大模型哪些参数可以被调用。包含两个部件。

- 参数描述：如果没有这部份，大模型就无法确定该输入什么参数。这些参数应当是被良好命名和描述的。
- 执行函数：一般是 **Python** 调用。

<div class="alert alert-info">
<b>注意思考</b><br>
    
- 必须为智能体提供可用的 **Tool**
- 必须为智能体提供良好的描述
</div>

## Toolkits

**langchain** 为很多通用任务准备了开箱即用的工具集，如：
- 生成或执行 **python** 代码 [https://python.langchain.com/docs/integrations/toolkits/python]
- 浏览器机器人 [https://python.langchain.com/docs/integrations/toolkits/playwright]
- 比较两个文档 [https://python.langchain.com/docs/integrations/toolkits/document_comparison_toolkit]
- 处理 **CSV** [https://python.langchain.com/docs/integrations/toolkits/csv]
- 处理 **pandas** [https://python.langchain.com/docs/integrations/toolkits/pandas]
- 处理 **SQL** [https://python.langchain.com/docs/integrations/toolkits/sql_database]
- 访问 **github** [https://python.langchain.com/docs/integrations/toolkits/github]
- 访问 **gitlab** [https://python.langchain.com/docs/integrations/toolkits/gitlab]
- 访问 **PowerBI** [https://python.langchain.com/docs/integrations/toolkits/powerbi]

# 常用工具

## Web请求

In [87]:
from langchain.agents import load_tools

requests_tools = load_tools(["requests_all"])
requests_tools

[RequestsGetTool(requests_wrapper=TextRequestsWrapper(headers=None, aiosession=None, auth=None, response_content_type='text')),
 RequestsPostTool(requests_wrapper=TextRequestsWrapper(headers=None, aiosession=None, auth=None, response_content_type='text')),
 RequestsPatchTool(requests_wrapper=TextRequestsWrapper(headers=None, aiosession=None, auth=None, response_content_type='text')),
 RequestsPutTool(requests_wrapper=TextRequestsWrapper(headers=None, aiosession=None, auth=None, response_content_type='text')),
 RequestsDeleteTool(requests_wrapper=TextRequestsWrapper(headers=None, aiosession=None, auth=None, response_content_type='text'))]

## 向量检索

In [ ]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langgraph_search",
    "搜索关于LangGraph的信息。询问任何关于LangGraph的信息，你都必须使用这个工具!",
)

## SQL

In [88]:
from langchain.tools.sql_database.tool import (
    BaseSQLDatabaseTool,
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)

## Shell

In [65]:
from langchain.tools import ShellTool

shell_tool = ShellTool()

In [67]:
print(shell_tool.run({"commands": ["echo 'Hello World!'", "time"]}))

Executing command:
 ["echo 'Hello World!'", 'time']
Hello World!

real	0m0.001s
user	0m0.000s
sys	0m0.000s



/Users/xuehongwei/Library/Caches/pypoetry/virtualenvs/langchain-book-u1j6yauo-py3.10/lib/python3.10/site-packages/langchain_community/tools/shell/tool.py:32: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


在智能体中使用：

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain.tools import ShellTool

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0)
prompt = hub.pull("hwchase17/openai-tools-agent")

shell_tool = ShellTool()
shell_tool.description = shell_tool.description + f"args {shell_tool.args}".replace(
    "{", "{{"
).replace("}", "}}")

self_ask_with_search = create_openai_tools_agent(
    llm, [shell_tool], prompt
)

agent_executor = AgentExecutor(agent=self_ask_with_search, tools=[shell_tool], verbose=True)
agent_executor.invoke(
    {"input": "下载网站 langchain.com 并提取10个URL，最后返回他们的列表"}
)

## 文件管理

In [60]:
from tempfile import TemporaryDirectory

from langchain_community.agent_toolkits import FileManagementToolkit

# We'll make a temporary directory to avoid clutter
working_directory = TemporaryDirectory()

指定根目录，然后选择所有工具：

In [61]:
toolkit = FileManagementToolkit(
    root_dir=str(working_directory.name)
)  # If you don't provide a root_dir, operations will default to the current working directory
toolkit.get_tools()

[CopyFileTool(root_dir='/var/folders/f5/rlf27f4n6wzc_k4x7y4vzm5h0000gn/T/tmpxzo9m_5m'),
 DeleteFileTool(root_dir='/var/folders/f5/rlf27f4n6wzc_k4x7y4vzm5h0000gn/T/tmpxzo9m_5m'),
 FileSearchTool(root_dir='/var/folders/f5/rlf27f4n6wzc_k4x7y4vzm5h0000gn/T/tmpxzo9m_5m'),
 MoveFileTool(root_dir='/var/folders/f5/rlf27f4n6wzc_k4x7y4vzm5h0000gn/T/tmpxzo9m_5m'),
 ReadFileTool(root_dir='/var/folders/f5/rlf27f4n6wzc_k4x7y4vzm5h0000gn/T/tmpxzo9m_5m'),
 WriteFileTool(root_dir='/var/folders/f5/rlf27f4n6wzc_k4x7y4vzm5h0000gn/T/tmpxzo9m_5m'),
 ListDirectoryTool(root_dir='/var/folders/f5/rlf27f4n6wzc_k4x7y4vzm5h0000gn/T/tmpxzo9m_5m')]

也可以选择部份工具：

In [62]:
tools = FileManagementToolkit(
    root_dir=str(working_directory.name),
    selected_tools=["read_file", "write_file", "list_directory"],
).get_tools()
tools

[ReadFileTool(root_dir='/var/folders/f5/rlf27f4n6wzc_k4x7y4vzm5h0000gn/T/tmpxzo9m_5m'),
 WriteFileTool(root_dir='/var/folders/f5/rlf27f4n6wzc_k4x7y4vzm5h0000gn/T/tmpxzo9m_5m'),
 ListDirectoryTool(root_dir='/var/folders/f5/rlf27f4n6wzc_k4x7y4vzm5h0000gn/T/tmpxzo9m_5m')]

用`write_tool`工具写入一个文件：

In [63]:
read_tool, write_tool, list_tool = tools
write_tool.invoke({"file_path": "example.txt", "text": "Hello World!"})

'File written successfully to example.txt.'

In [64]:
list_tool.invoke({})

'example.txt'

## 人类输入

In [85]:
from langchain.tools import HumanInputRun

HumanInputRun().invoke("hello")



hello


''

## Python 代码执行

In [53]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

In [54]:
python_repl = PythonREPL()

In [56]:
python_repl.run("print(1+1)")

Python REPL can execute arbitrary code. Use with caution.


'2\n'

In [57]:
# You can create the tool to pass to an agent
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)

In [58]:
repl_tool.invoke("print(1+1)")

'2\n'

# 智能体类型

智能体可以按这些维度分类：
- 预期的模型是对话模型还是单次模型
- 是否支持多参数输入的工具
- 是否支持工具的并发调度
- 是否需要模型的附加参数

当前**langchain**主要包括的智能体类型有：

- **ReAct Agent**：是一个简单的Agent，适用于简单的模型。它使用ReAct框架来确定使用哪个工具，并且不支持多输入工具。适用于简单的问题和任务。
- **Self Ask With Search Agent**：是一个简单的Agent，适用于简单的模型和只有一个搜索工具的情况。它将复杂的问题分解为一系列简单的问题，并使用搜索工具查找答案。适用于需要使用搜索工具查找答案的问题。
- **OpenAI Tools Agent**：是一个适用于Chat模型的Agent，支持聊天历史和多输入工具。它可以与最新的OpenAI模型一起使用，并且可以调用多个工具并行执行任务。适用于需要使用多个工具并且需要聊天历史的任务。
- **OpenAI Functions Agent**：是一个适用于Chat模型的Agent，支持聊天历史和多输入工具。它专门针对使用OpenAI函数的模型进行了优化，并且可以调用多个函数并行执行任务。适用于使用OpenAI模型或经过微调以支持函数调用的开源模型的任务。
- **XML Agent**：是一个使用XML标记的Agent，适用于处理XML格式的工具输入和输出。适用于需要处理XML数据的任务。
- **JSON Agent**：是一个使用JSON格式的Agent，适用于处理JSON格式的工具输入和输出。适用于需要处理JSON数据的任务。
- **Structured Chat Agent**：是一个适用于Chat模型的Agent，支持聊天历史和多输入工具。它可以处理具有多个输入的工具，并支持更复杂的工具使用场景，如精确导航浏览器。适用于需要处理具有多个输入的工具的任务。

其中：**OpenAI Tools Agent** 需要与最新的OpenAI模型一起使用，因为它利用了OpenAI的最新功能，即并行函数调用（tool calling）。
这种功能允许模型一次性返回多个函数调用结果，从而提高了效率和性能。
因此，为了充分利用**OpenAI Tools Agent**的功能，需要使用支持并行函数调用的最新OpenAI模型。

## ReAct Agent 示例

### 定义智能体

In [ ]:
!poetry add langchainhub

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=5)]

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_openai import OpenAI

prompt = hub.pull("hwchase17/react")
llm = OpenAI()
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### 运行智能体

In [ ]:
agent_executor.invoke({"input": "吴京的老婆是谁？"})

In [ ]:
agent_executor.invoke({"input": "吴京的老婆主持过什么综艺？"})

### 增加历史消息支持

In [ ]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react-chat")

In [ ]:
# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage

agent_executor.invoke(
    {
        "input": "what's my name? Only use a tool if needed, otherwise respond with Final Answer",
        # Notice that chat_history is a string, since this prompt is aimed at LLMs, not chat models
        "chat_history": "Human: Hi! My name is Bob\nAI: Hello Bob! Nice to meet you",
    }
)

## OpenAI Tools Agent 示例

### 定义智能体

In [ ]:
!poetry add langchain-openai tavily-python

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults
tools = [TavilySearchResults(max_results=2)]

In [91]:
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

prompt = hub.pull("hwchase17/openai-tools-agent")
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)
agent = create_openai_tools_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### 执行智能体

In [ ]:
agent_executor.invoke({"input": "what is LangChain?"})

### 支持消息历史

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage

agent_executor.invoke(
    {
        "input": "what's my name? Don't use tools to look this up unless you NEED to",
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
    }
)

## Self-ask with search 示例

该Agent的实际效果似乎取决于大模型的能力，使用GPT4明显要优于GPT3.5版本。

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor, create_self_ask_with_search_agent
from langchain_openai import OpenAI
from langchain_community.tools.tavily_search import TavilyAnswer

In [ ]:
# 下载一个模板
prompt = hub.pull("hwchase17/self-ask-with-search")
print(prompt.template)

In [ ]:
# Choose the LLM that will drive the agent
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model = "gpt-4-1106-preview")
# llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

search = TavilyAnswer(max_results=3)

from langchain.agents import create_self_ask_with_search_agent
# tools = [
#     Tool(
#         name="Intermediate Answer",
#         func=search.run,
#         description="useful for when you need to ask with search.",
#     )
# ]
tools = [TavilyAnswer(max_results=1, name="Intermediate Answer")]
agent = create_self_ask_with_search_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input": "吴京的老婆主持过哪些综艺？"}, verbose = True, handle_parsing_errors=True)

In [ ]:
agent_executor.invoke({"input": "马斯克的哪家公司最赚钱？"}, verbose = True)